<a href="https://colab.research.google.com/github/abirpy/NLP-Projects/blob/master/distilbert_base_uncased_finetuned_emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers[sentencepiece]

     |████████████████████████████████| 306 kB 9.8 MB/s 
     |████████████████████████████████| 3.4 MB 62.6 MB/s 
     |████████████████████████████████| 132 kB 76.2 MB/s 
     |████████████████████████████████| 61 kB 680 kB/s 
     |████████████████████████████████| 243 kB 78.7 MB/s 
     |████████████████████████████████| 1.1 MB 58.8 MB/s 
     |████████████████████████████████| 160 kB 56.2 MB/s 
     |████████████████████████████████| 192 kB 66.3 MB/s 
     |████████████████████████████████| 271 kB 77.6 MB/s 
     |████████████████████████████████| 3.3 MB 70.5 MB/s 
     |████████████████████████████████| 895 kB 70.5 MB/s 
     |████████████████████████████████| 596 kB 76.3 MB/s 
     |████████████████████████████████| 1.2 MB 62.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install huggingface

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

from datasets import load_dataset

dataset = load_dataset("emotion")

Downloading:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

Using custom data configuration default


Downloading:   0%|          | 0.00/1.66M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/204k [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset emotion downloaded and prepared to /root/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [5]:
import pandas as pd
import numpy as np

In [6]:
arr = pd.Series(dataset['train']['label'])
arr.value_counts()

1    5362
0    4666
3    2159
4    1937
2    1304
5     572
dtype: int64

In [7]:
label_names = dataset['train'].features['label'].names
label_names

['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

(0 = negative, 2 = neutral, 4 = positive)

In [8]:
id2label = {k:v for k,v in enumerate(label_names)}
label2id = {v:k for k,v in id2label.items()}

In [9]:
id2label

{0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}

In [10]:
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [11]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [12]:
tokenized_datasets['train']

Dataset({
    features: ['attention_mask', 'input_ids', 'label', 'text', 'token_type_ids'],
    num_rows: 16000
})

In [13]:
tokenized_datasets = tokenized_datasets.remove_columns(['text'])
tokenized_datasets["train"].column_names

['attention_mask', 'input_ids', 'label', 'token_type_ids']

In [14]:
tokenized_datasets['train'][0]

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1],
 'input_ids': [101, 178, 1238, 1204, 1631, 21820, 21896, 102],
 'label': 0,
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0]}

In [15]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint,
    id2label = id2label,
    label2id = label2id,
)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [16]:
model.config.num_labels

6

In [17]:
from transformers import TrainingArguments

args = TrainingArguments(
    "distilbert-base-uncased-finetuned-emotion",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    per_device_train_batch_size = 64,
    per_device_eval_batch_size = 64,
    push_to_hub=True,
)

In [18]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/token.
        (Deprecated, will be removed in v0.3.0) To login with username and password instead, interrupt with Ctrl+C.
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine

In [19]:
!git config --global credential.helper store

In [20]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 1s (1,612 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 155225 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting up git-lfs (2.3.4-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [ ]:
!pip install bert_score

In [26]:
from datasets import load_metric
def compute_metrics(eval_preds):
    metric = load_metric("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [27]:
from transformers import Trainer

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/content/distilbert-base-uncased-finetuned-emotion is already a clone of https://huggingface.co/Tahsin/distilbert-base-uncased-finetuned-emotion. Make sure you pull the latest changes with `repo.git_pull()`.


In [28]:
trainer.train()

***** Running training *****
  Num examples = 16000
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 750


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.163477,0.929500
2,0.111000,0.151523,0.936000
3,0.111000,0.156134,0.928500


***** Running Evaluation *****
  Num examples = 2000
  Batch size = 64


Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Saving model checkpoint to distilbert-base-uncased-finetuned-emotion/checkpoint-250
Configuration saved in distilbert-base-uncased-finetuned-emotion/checkpoint-250/config.json
Model weights saved in distilbert-base-uncased-finetuned-emotion/checkpoint-250/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-emotion/checkpoint-250/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-emotion/checkpoint-250/special_tokens_map.json
tokenizer config file saved in distilbert-base-uncased-finetuned-emotion/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-emotion/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 64
Saving model checkpoint to distilbert-base-uncased-finetuned-emotion/checkpoint-500
Configuration saved in distilbert-base-uncased-finetuned-emotion/checkpoint-500/config.json
Model weights saved in distilbert-base-uncased-finetuned-emotion/chec

TrainOutput(global_step=750, training_loss=0.09765429178873698, metrics={'train_runtime': 552.7467, 'train_samples_per_second': 86.839, 'train_steps_per_second': 1.357, 'total_flos': 1441900366479360.0, 'train_loss': 0.09765429178873698, 'epoch': 3.0})

In [29]:
trainer.push_to_hub(commit_message="Training complete")

Saving model checkpoint to distilbert-base-uncased-finetuned-emotion
Configuration saved in distilbert-base-uncased-finetuned-emotion/config.json
Model weights saved in distilbert-base-uncased-finetuned-emotion/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-emotion/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-emotion/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.36k/413M [00:00<?, ?B/s]

Upload file runs/Jan06_07-18-26_624ebc275b21/events.out.tfevents.1641454123.624ebc275b21.74.3:  67%|######7   …

To https://huggingface.co/Tahsin/distilbert-base-uncased-finetuned-emotion
   ff2a148..5d609ee  main -> main

To https://huggingface.co/Tahsin/distilbert-base-uncased-finetuned-emotion
   5d609ee..ba9481e  main -> main



'https://huggingface.co/Tahsin/distilbert-base-uncased-finetuned-emotion/commit/5d609ee77e95974b185b7ea2cd495081366b8037'

In [ ]:
from transformers import pipeline
mood = pipeline('sentiment-analysis', model='Tahsin/distilbert-base-uncased-finetuned-emotion')
mood("I love my country")